### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### GOOGLE SHEETS

In [3]:
comp = descarga('1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw','Competencias')

Descarga Correcta!


In [4]:
comp_type = {}
for i in comp['Country'].unique():
    comp_type[i] = comp[comp['Country'] == i]['Tipo'].unique()[0]

### QUERIES

In [5]:
q_crudo = '''WITH acq_table AS (
    SELECT o.user.id AS user
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    WHERE o.registered_date >= DATE('2021-06-01')
          AND o.is_acquisition = 1),
    users_table AS (
    SELECT o.user.id AS user,
           CASE WHEN acq.user IS NOT NULL THEN TRUE ELSE FALSE END AS new_user,
           cn.country_name AS country,
           SUM(o.shipping_amount_no_discount + amount_no_discount) AS total_amount,
           SUM(o.voucher_discount_amount) AS voucher_dc,
           IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) AS adr,
           CASE WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.1 THEN 'Under 10'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.2 THEN 'Under 20'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.3 THEN 'Under 30'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.4 THEN 'Under 40'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.5 THEN 'Under 50'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.6 THEN 'Under 60'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.7 THEN 'Under 70'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.8 THEN 'Under 80'
                WHEN IFNULL(SAFE_DIVIDE(SUM(o.voucher_discount_amount),SUM(o.shipping_amount_no_discount + amount_no_discount)),0) < 0.9 THEN 'Under 90'
                ELSE 'Over 90' END AS adr_cluster
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_user` AS u ON o.user.id = u.user_id
    INNER JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id AND cn.active
    LEFT JOIN acq_table AS acq ON o.user.id = acq.user
    WHERE o.registered_date >= DATE_ADD(CURRENT_DATE(),INTERVAL -181 DAY)
        AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3),
    cluster_table AS (
    SELECT ut.country AS country,
           ut.new_user AS new_user,
           ut.adr_cluster AS adr_cluster,
           COUNT(DISTINCT ut.user) AS cluster_users
    FROM users_table AS ut
    GROUP BY 1,2,3),
    orders_table AS (
    SELECT o.user.id AS user,
           o.order_id AS order_id,
           o.registered_date AS fecha,
           cn.country_name AS country,
           IFNULL(c.city_name,'-') AS city,
           IFNULL(a.area_name,'-') AS area,
           CASE WHEN tc.order_id IS NOT NULL THEN TRUE ELSE FALSE END AS with_voucher,
           tca.talon_campaign_name AS campaign,
           o.total_amount_with_delivery_costs AS gmv,
           CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN TRUE ELSE FALSE END AS shopper,
           CASE WHEN tc.order_id IS NOT NULL THEN tc.coupon_used_amount ELSE NULL END AS used_amount
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
    WHERE o.registered_date BETWEEN DATE('2021-05-01') AND DATE('2021-08-31')
          AND o.order_status = 'CONFIRMED')
SELECT ut.country AS Country,
       ut.adr_cluster AS ADR_Cluster,
       CASE WHEN ct.new_user THEN 'New' ELSE 'Old' END AS User_Type, 
       ct.cluster_users AS Cluster_Users,
       ot.city AS City,
       ot.area AS Area,
       IFNULL(ot.campaign,'-') AS Campaign,
       FORMAT_DATE('%Y-%m',ot.fecha) AS Month,
       COUNT(DISTINCT ot.order_id) AS Total_Orders,
       COUNT(DISTINCT CASE WHEN ot.with_voucher THEN ot.order_id ELSE NULL END) AS Voucher_Orders,
       SUM(ot.gmv) AS GMV,
       IFNULL(SUM(CASE WHEN ot.shopper = FALSE THEN ot.used_amount ELSE 0 END),0) AS Amount,
       IFNULL(SUM(CASE WHEN ot.shopper THEN ot.used_amount ELSE 0 END),0) AS Amount_Shopper,
FROM users_table AS ut
LEFT JOIN cluster_table AS ct ON ut.country = ct.country AND ut.adr_cluster = ct.adr_cluster AND ut.new_user = ct.new_user
LEFT JOIN orders_table AS ot ON ut.user = ot.user AND ut.country = ot.country
WHERE ut.country IN ('Argentina','Chile')
GROUP BY 1,2,3,4,5,6,7,8
ORDER BY 1,2'''

In [6]:
# Descargo la data
bq_crudo = pd.io.gbq.read_gbq(q_crudo, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 930458/930458 [03:16<00:00, 4739.24rows/s]


In [7]:
# Copio las bases
crudo = bq_crudo.copy()

### TRABAJO

In [8]:
# Trabajo Crudo
cols = ['Country','ADR_Cluster','City','Area','User_Type','Campaign','Month']
val = [i for i in crudo.columns if i not in cols]
crudo[val] = crudo[val].astype(float)
crudo[cols] = crudo[cols].apply(lambda x: x.astype(str).str.upper())
crudo['Campaign'] = crudo['Campaign'].str.replace(' ', '')

In [9]:
# Trabajo CRM
crudo['Type'] = crudo.apply(lambda x: filtros(x['Campaign'],'TIPOS'),axis=1)
crudo['Type'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Type'],axis=1)
crudo['Segment'] = crudo['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
crudo['Segment'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Segment'],axis=1)
crudo['Level'] = crudo['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
crudo['Level'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Level'],axis=1)
crudo['Campaña'] = crudo['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
crudo['Campaña'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Campaña'],axis=1)
crudo['Automated'] = crudo['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
crudo['Automated'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Automated'],axis=1)
crudo['Budget'] = crudo['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
crudo['Budget'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Budget'],axis=1)
crudo['Trial'] = crudo['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
crudo['Trial'] = crudo.apply(lambda x: '-' if x['Campaign'] == '-' else x['Trial'],axis=1)
# Coloco el IVA
crudo = func_iva(crudo)
# Marco las campañas a filtrar
crudo['Filtrar'] = crudo['Campaign'].apply(filtrar_cam)
# Saco el IVA
used = 'Amount'
used_shopper = 'Amount_Shopper'
crudo['Final'] = crudo[used_shopper] / (1 + crudo['IVA']) * crudo['Extra'] + crudo[used]

In [10]:
# Filtro las campañas de Voucher Campaigns
crudo['Tipo'] = crudo['Type'].apply(lambda x: '-' if x == 'VOUCHER CAMPAIGNS' else 'OK')
crudo['Budget'] = crudo['Budget'].apply(lambda x: '-' if x in ['DMARTS','NONE','-'] else x)

In [11]:
# Coloco el tipo de Competencia
crudo['Competitive Type'] = crudo['Country'].apply(lambda x: comp_type[x] if x in list(comp_type.keys()) else '-')
crudo['Union'] = crudo.apply(lambda x: str(x['Country'])+'-'+str(x['City'])+'-'+str(x['Area']) if x['Competitive Type'] == 'CITY-AREA' else str(x['Country'])+'-'+str(x['City'])+'-',axis=1)
# Hago un merge con la competencia
crudo = crudo.merge(comp[['Union','Competencia']],on=['Union'],how='left')
crudo['Competencia'].replace([np.nan,np.inf,-np.inf],'LOW',inplace=True)
# Marco pais sin Competencia
crudo['Pais Competido'] = crudo['Country'].apply(lambda x: 'SI' if x in list(comp_type.keys()) else 'NO')

In [12]:
# Creo la PT
index = ['Country','Month','ADR_Cluster','User_Type','Competencia','Budget','Filtrar','Tipo']
values = ['Cluster_Users','Total_Orders','Voucher_Orders','GMV','Final']
aggfunc = {'Cluster_Users':'mean',
           'Total_Orders':'sum',
           'Voucher_Orders':'sum',
           'GMV':'sum',
           'Final':'sum'}
final = crudo.pivot_table(index=index,values=values,aggfunc=aggfunc,fill_value=0).reset_index()

### CARGA

In [20]:
carga(final,'1gVjnzSHHAGwbK6_QxvRWlehF0W28F6w2h9ZmBbhazQ8','Crudo')

Carga Correcta!
